##### Import modules #####

In [1]:
from nltk.corpus import stopwords

In [20]:
import pandas as pd
import heapq
import os  
import sys  
import json
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from heapq import heappush, nlargest

In [11]:
dataset = pd.read_csv('books_dataset.tsv', sep='\t', index_col=None)

In [12]:
dataset.head(5)

,title,series,author,ratingValue,ratingCount,plots,reviewCount,date,characters,settings,url,index
0,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,6414062.0,"Could you survive on your own in the wild, wit...",374.0,September 14th 2008,"Katniss Everdeen, Peeta Mellark, Cato (Hunger ...","District 12 Panem, Capitol Panem, Panem",https://www.goodreads.com/book/show/2767052-th...,1
1,Harry Potter and the Order of the Phoenix,Harry Potter #5,J.K. Rowling,4.50,2528402.0,There is a door at the end of a silent corrido...,870.0,September 2004,"Sirius Black, Draco Malfoy, Ron Weasley, Petun...","Hogwarts School of Witchcraft and Wizardry, Lo...",https://www.goodreads.com/book/show/2.Harry_Po...,2
2,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,4532078.0,The unforgettable novel of a childhood in a sl...,324.0,May 23rd 2006,"Scout Finch, Atticus Finch, Jem Finch, Arthur ...",Maycomb Alabama,https://www.goodreads.com/book/show/2657.To_Ki...,3
3,Pride and Prejudice,NaN,Jane Austen,4.26,3021524.0,"Since its immediate success in 1813, Pride and...",279.0,October 10th 2000,"Mr. Bennet, Mrs. Bennet, Jane Bennet, Elizabet...","United Kingdom, Derbyshire England, England, H...",https://www.goodreads.com/book/show/1885.Pride...,4
4,Twilight,The Twilight Saga #1,Stephenie Meyer,3.60,4994637.0,About three things I was absolutely positive.F...,501.0,September 6th 2006,"Edward Cullen, Jacob Black, Laurent, Renee, Be...","Forks Washington, Phoenix Arizona, Washington ...",https://www.goodreads.com/book/show/41865.Twil...,5


In [13]:
df = dataset[['title','plots','author','characters','settings','url','index']]
df.head(5)

,title,plots,author,characters,settings,url,index
0,The Hunger Games,"Could you survive on your own in the wild, wit...",Suzanne Collins,"Katniss Everdeen, Peeta Mellark, Cato (Hunger ...","District 12 Panem, Capitol Panem, Panem",https://www.goodreads.com/book/show/2767052-th...,1
1,Harry Potter and the Order of the Phoenix,There is a door at the end of a silent corrido...,J.K. Rowling,"Sirius Black, Draco Malfoy, Ron Weasley, Petun...","Hogwarts School of Witchcraft and Wizardry, Lo...",https://www.goodreads.com/book/show/2.Harry_Po...,2
2,To Kill a Mockingbird,The unforgettable novel of a childhood in a sl...,Harper Lee,"Scout Finch, Atticus Finch, Jem Finch, Arthur ...",Maycomb Alabama,https://www.goodreads.com/book/show/2657.To_Ki...,3
3,Pride and Prejudice,"Since its immediate success in 1813, Pride and...",Jane Austen,"Mr. Bennet, Mrs. Bennet, Jane Bennet, Elizabet...","United Kingdom, Derbyshire England, England, H...",https://www.goodreads.com/book/show/1885.Pride...,4
4,Twilight,About three things I was absolutely positive.F...,Stephenie Meyer,"Edward Cullen, Jacob Black, Laurent, Renee, Be...","Forks Washington, Phoenix Arizona, Washington ...",https://www.goodreads.com/book/show/41865.Twil...,5


In [14]:
df.isnull().sum()

title             1
plots             0
author            1
characters    11070
settings      12235
url               2
index             0
dtype: int64

In [15]:
df['title'].fillna('', inplace=True)
df['url'].fillna('', inplace=True)
df['author'].fillna('', inplace=True)
df['characters'].fillna('', inplace=True)
df['settings'].fillna('', inplace=True)

C:\Users\Sanduni\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [8]:
df.isnull().sum()

title         0
plots         0
author        0
characters    0
settings      0
url           0
index         0
dtype: int64

In [16]:
stop_words = set(stopwords.words('english'))
def remove_stop_word(phrases):
    tokenizer = RegexpTokenizer(r'[a-z]+')
    word = tokenizer.tokenize(phrases.lower())
    return [w for w in word if w not in stop_words]

In [17]:
df['title'] = df.apply(lambda x : remove_stop_word(x['title']), axis=1)
df['plots'] = df.apply(lambda x : remove_stop_word(x['plots']), axis=1)
df['author'] = df.apply(lambda x : remove_stop_word(x['author']), axis=1)
df['characters'] = df.apply(lambda x : remove_stop_word(x['characters']), axis=1)
df['settings'] = df.apply(lambda x : remove_stop_word(x['settings']), axis=1)

<ipython-input-17-5473df57246b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'] = df.apply(lambda x : remove_stop_word(x['title']), axis=1)
<ipython-input-17-5473df57246b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['plots'] = df.apply(lambda x : remove_stop_word(x['plots']), axis=1)
<ipython-input-17-5473df57246b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [18]:
words=[] 
all_words = [] 
for i in range(len(df)):
    c_all = []
    for wordt in df['title'][i]:
        words.append(wordt)
        c_all.append(wordt)
    for wordp in df['plots'][i]:
        words.append(wordp)
        c_all.append(wordp)
    for worda in df['author'][i]:
        words.append(worda)
        c_all.append(worda)
    for wordc in df['characters'][i]:
        words.append(wordc)
        c_all.append(wordc)
    for wordst in df['settings'][i]:
        words.append(wordst)
        c_all.append(wordst)
    all_words .append(c_all)
df["All Words"] = pd.Series(all_words)
df.head(5)

<ipython-input-18-1ca1c0426f1d>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["All Words"] = pd.Series(all_words)


,title,plots,author,characters,settings,url,index,All Words
0,"[hunger, games]","[could, survive, wild, every, one, make, sure,...","[suzanne, collins]","[katniss, everdeen, peeta, mellark, cato, hung...","[district, panem, capitol, panem, panem]",https://www.goodreads.com/book/show/2767052-th...,1,"[hunger, games, could, survive, wild, every, o..."
1,"[harry, potter, order, phoenix]","[door, end, silent, corridor, haunting, harry,...","[j, k, rowling]","[sirius, black, draco, malfoy, ron, weasley, p...","[hogwarts, school, witchcraft, wizardry, londo...",https://www.goodreads.com/book/show/2.Harry_Po...,2,"[harry, potter, order, phoenix, door, end, sil..."
2,"[kill, mockingbird]","[unforgettable, novel, childhood, sleepy, sout...","[harper, lee]","[scout, finch, atticus, finch, jem, finch, art...","[maycomb, alabama]",https://www.goodreads.com/book/show/2657.To_Ki...,3,"[kill, mockingbird, unforgettable, novel, chil..."
3,"[pride, prejudice]","[since, immediate, success, pride, prejudice, ...","[jane, austen]","[mr, bennet, mrs, bennet, jane, bennet, elizab...","[united, kingdom, derbyshire, england, england...",https://www.goodreads.com/book/show/1885.Pride...,4,"[pride, prejudice, since, immediate, success, ..."
4,[twilight],"[three, things, absolutely, positive, first, e...","[stephenie, meyer]","[edward, cullen, jacob, black, laurent, renee,...","[forks, washington, phoenix, arizona, washingt...",https://www.goodreads.com/book/show/41865.Twil...,5,"[twilight, three, things, absolutely, positive..."


In [19]:
words=set(words)
words = list(words)
vocab={}
for i in range(len(words)):
    vocab.update({words[i] : i })
with open("vocabulary.json", "w", encoding = "utf8") as v:
    v.write(json.dumps(vocab))
    
for i in range(len(df)):
    for j in range(len(df["All Words"][i])):
        if df["All Words"][i][j] in vocab.keys():
            df["All Words"][i][j] = vocab[df["All Words"][i][j]]

In [25]:
from collections import Counter,defaultdict
df["All Words"] = df["All Words"].apply(lambda x : Counter(x))

<ipython-input-25-d382b67d5cd4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["All Words"] = df["All Words"].apply(lambda x : Counter(x))


In [22]:
def Term_Frequency(doc_words):
    bow = 0
    for k, v in doc_words.items():
        bow = bow + v
    tf_word = {}
    for word, count in doc_words.items():
        tf_word[word] = count / float(bow)
    return tf_word

In [24]:
TF = []
for i in range(len(df["All Words"])):
    TF.append(Term_Frequency(df["All Words"][i]))
df["TF"] = TF

<ipython-input-24-c4098c20c2ae>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["TF"] = TF


In [27]:
inverted_index_new =  defaultdict(list)
for i in range(len(df)):
    for keys,values in df['All Words'][i].items():
        inverted_index_new[keys].append(i)

In [28]:
with open('inverted_index_new.json', 'w') as fp:
    json.dump(inverted_index_new, fp)

In [29]:
import math
idf = {}
for k, v in vocab.items():
    idf[v] = math.log(len(df["All Words"])/len(inverted_index_new[v]))
with open("idf.json", "w", encoding = "utf8") as i_d_f:
    i_d_f.write(json.dumps(idf))

In [30]:
def tf_idf(docid, termid):
    return((df["All Words"][docid][termid]*idf[termid])/sum(df["All Words"][docid]))

In [31]:
df.head(1)

,title,plots,author,characters,settings,url,index,All Words,TF
0,"[hunger, games]","[could, survive, wild, every, one, make, sure,...","[suzanne, collins]","[katniss, everdeen, peeta, mellark, cato, hung...","[district, panem, capitol, panem, panem]",https://www.goodreads.com/book/show/2767052-th...,1,"{3204: 6, 73782: 7, 25284: 1, 61219: 1, 22863:...","{3204: 0.043478260869565216, 73782: 0.05072463..."


In [32]:
norm = []
for i in range(len(df)):
    s = 0
    for word in df["All Words"][i].keys():
        s = s + (df["TF"][i][word]*idf[word])**2
    norm.append(s**(1/2))
df["Norm"] = norm

<ipython-input-32-71b3cd48d360>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Norm"] = norm


In [33]:
df.head(1)

,title,plots,author,characters,settings,url,index,All Words,TF,Norm
0,"[hunger, games]","[could, survive, wild, every, one, make, sure,...","[suzanne, collins]","[katniss, everdeen, peeta, mellark, cato, hung...","[district, panem, capitol, panem, panem]",https://www.goodreads.com/book/show/2767052-th...,1,"{3204: 6, 73782: 7, 25284: 1, 61219: 1, 22863:...","{3204: 0.043478260869565216, 73782: 0.05072463...",0.660595


In [34]:
inverted_index_2 = defaultdict(list)
for k, v in inverted_index_new.items():
    for l in v:
        inverted_index_2[k].append((l, tf_idf(l,k)/df["Norm"][l]))
        
with open("inverted_index_2.json", "w", encoding = "utf8") as i_d_2:
    i_d_2.write(json.dumps(inverted_index_2))

In [38]:
def SearchEngine2(dataset):
    query = input()
    if query == "":
        return("Please input again!")
    query=query.split(" ")
    books = list(dataset.index)
    books = set(books)

    vocab=json.loads(open("vocabulary.json").read())
    inverted_index_new = json.loads(open("inverted_index_new.json").read())
    inverted_index_2=json.loads(open("inverted_index_2.json").read())
    Inverse_Document_Frequency = json.loads(open("idf.json").read())
    # Query whether the search keyword exists in the vocabulary
    for word in query:
        if word in vocab.keys():
            query[query.index(word)]=(vocab[word])
        else:
            print("word Not found")
    query_vec = {}
    #Compute tfidf
    for item in query:
        query_vec[item]=  Term_Frequency(Counter(query))[item]*Inverse_Document_Frequency[str(item)]

    for word in query: 
        books = books.intersection(inverted_index_new[str(word)])
    dataset = dataset.drop(index=dataset.index.difference(books))
    
    
    doc_vec = {}
    
    for book in books:
        vec = []
        for word_ in query:
            index = inverted_index_2[str(word_)]
            for i in range(len(index)):
                if index[i][0] == book:
                    vec.append(index[i][1])
        doc_vec[book] = vec
    
    def dot(vector_1, vector_2):
        sum = 0
        for i in range(len(vector_1)):
            sum = sum + vector_1[i]*vector_2[i]
        return(sum)
    def norm(vector):
        if len(vector) > 1:
            add = 0
            for i in range(len(vector)):
                add = add + vector[i]**2
            return add**(1/2)
        else:
            return(vector)
    # Compute Cosine similarity
    similarity = {}
    for book in books:
        if isinstance(norm(list(query_vec.values())), float):
            similarity[book] = dot(list(query_vec.values()),doc_vec[book])/(norm(list(query_vec.values()))*norm(doc_vec[book]))
        else:
            similarity[book] = dot(list(query_vec.values()),doc_vec[book])/(norm(list(query_vec.values()))[0]*norm(doc_vec[book]))

    the_col = []
    for i in range(len(dataset)):
        for k, v in similarity.items():
            if dataset.index[i] == k:
                the_col.append(similarity[k])

    dataset["Similarity"] = the_col
    truth = []
    for book_d in books:
        if dataset["Similarity"][book_d] in heapq.nlargest(5,dataset["Similarity"]):
            truth.append(book_d)
    newdataset = dataset.drop(index=dataset.index.difference(truth))
    return(newdataset[['title','plots','url','Similarity']].sort_values(by = 'Similarity',axis = 0,ascending = False))

In [39]:
SearchEngine2(dataset)

harry door


,title,plots,url,Similarity
12885,Sheet Music: The Chronicles of Narnia - Prince...,(Piano/Vocal/Guitar Songbook). All nine songs ...,https://www.goodreads.com/review/show/123110062,0.954600
1,Harry Potter and the Order of the Phoenix,There is a door at the end of a silent corrido...,https://www.goodreads.com/book/show/2.Harry_Po...,0.859881
2606,Grave Peril,An alternative cover edition with a different ...,https://www.goodreads.com/review/show/133867511,0.845448


## Point 3: New Score! ##

In [43]:
def new_score():
    query = input()
    if query == "":
        return("Please input again!")
    query=query.split(" ")
    books = list(dataset.index)
    books = set(books)

    vocab=json.loads(open("vocabulary.json").read())
    inverted_index_new = json.loads(open("inverted_index_new.json").read())
    inverted_index_2=json.loads(open("inverted_index_2.json").read())
    Inverse_Document_Frequency = json.loads(open("idf.json").read())
    # Query whether the search keyword exists in the vocabulary
    for word in query:
        if word in vocab.keys():
            query[query.index(word)]=(vocab[word])
        else:
            print("word Not found")
        query_vec = {}
    #Compute tfidf
    for item in query:
        query_vec[item]=  Term_Frequency(Counter(query))[item]*Inverse_Document_Frequency[str(item)]

    for word in query: 
        books = books.intersection(inverted_index_new[str(word)])
    dataset = dataset.drop(index=dataset.index.difference(books))
    
    
    doc_vec = {}
    
    for book in books:
        vec = []
        for word_ in query:
            index = inverted_index_2[str(word_)]
            for i in range(len(index)):
                if index[i][0] == book:
                    vec.append(index[i][1])
        doc_vec[book] = vec
            
    # Get exrtra information from the user to create a new score
    print("What is the name of the author of the book you looking for?:")
    ex_query_1 = input()
    print("What is the book rating you prefer?:")
    ex_query_2 = input()
    
    #Calculate new score
    score = {}
    for book in books:
        if dataset["bookAuthors"][book] == ex_query_1:
            auth = 0.5
        else:
            auth = 0
        if dataset["ratingValue"][book] >= 4.5:
            rt = 0.45
        elif dataset["ratingValue"][book] >= ex_query_2:
            rt = 0.30
        elif dataset["ratingValue"][book] >= 3.0:
            rt = 0.30
        elif dataset["ratingValue"][book] >= 1.5:
            rt = 0.10
        else:
            rt = 0.05
        score[book] = auth + rt   
                   
    score = []
    for i in range(len(dataset)):
        for k, v in score.items():
            if dataset.index[i] == k:
                score.append(score[k])

    dataset["New_Score"] = score
    truth = []
    for book in books:
        if dataset["New_Score"][book] in heapq.nlargest(5,dataset["New_Score"]):
            truth.append(book)
    newdataset = dataset.drop(index=dataset.index.difference(truth))
    return(newdataset[['title','plots','url','New_Score']].sort_values(by = 'New_Score',axis = 0,ascending = False))